<a href="https://colab.research.google.com/github/Mostafa-Rawash/get_facebook_interests/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import requests
import json
from urllib.parse import quote
import threading  
from pymongo import MongoClient

In [50]:
# make unterest have one object for loop on it in he database is empty
all_interest_list =[{"id": "6003956183466", "name": "Listening to music", "type": "null", "path": "null", "description": "null", "source": "null", "partner": "null", "audience_size": 3839630, "country": "null", "country_access": "null", "topic": "null"}]
done_list = []


# FAceBokk URL init.

# select the type of request adinterestsuggestion => to get suggestion around ur word 
reqType = 'adinterestsuggestion' 
# this Acces Token will be Authorized for one month and to get new one will use this request:=>  "https://graph.facebook.com/v9.0/oauth/access_token?grant_type=fb_exchange_token&client_id=<APP ID>&client_secret=<APP secret code>&fb_exchange_token=<short_Access-Token>"
# short_Access-Token come from :=> "https://developers.facebook.com/tools/explorer/"
access_token = "EAADKRQU7abABAEDq1OHRWeocZCY1PKYA0C0fuBtHUiZA7d3Ln6uqjoyohUDFNKGiZCPu0KQivzbsfncxWe4HZBTggZAME62fYkfKMp1npKZBixG98WpeBbFZBbyrNDKXn6IBTW15dYOKE5ZCMabebZCDLCvdaqCvsRi6N5WqmfTURqwZDZD"

#  Mongo DB init.

Mongo_userName= '<mongo user name>'
Mongo_password = '<mongo user password>'
client = MongoClient('<write here your mongo atlas connection link>')
db = client."<the data base name>"
db = client.get_database("<the data base name>")
# db = client.get_database("interests")
interstdb = db.'<collection name>'

In [46]:
# will red data from database to work in it
def get_data():
  global done_list
  global all_interest_list 
  for item in interstdb.find():
    del(item["_id"])        # remove the "_id" that generted by deafult with mongoosDB and remove it make the comper between this data and new one 
    all_interest_list.append(item) 
    print(len(all_interest_list)) 
  for item in donedb.find():
    # del(item["_id"])        # remove the "_id" that generted by deafult with mongoosDB and remove it make the comper between this data and new one 
    done_list.append(item["name"])   # item["data"]   come as array 
 
 

#  this function for delete repeated item in database
def cleaning_collection(collection_name):  
  collectionData = []
  x = 0
  for item in collection_name.find():
      del(item["_id"])
      if item not in collectionData:
        collectionData.append(item)
  collection_name.delete_many({})
  print(len(collectionData))
  collection_name.insert_many(collectionData)  
 
 
#   this function for append chunk_interests & chunk_done that come generate after 1000 request  
def append(c_i , c_d):
          if len(c_i) > 0:
            interstdb.insert_many(c_i)
            c_i = []
          print(" the c_i is added")
          if len(c_d) > 0:
            donedb.insert_many(c_d)            
            c_d= []
          print(" the c_d is added")
          print("END")
 
 
def main():
    chunk_intersts = []
    chunk_done = []
    count = int(0)
    for item in all_interest_list:
        if count < 1000: # to save the data evry 10000 loop 
            if item["name"] not in done_list: # to know in the name of interest that come from "all_interest_list" make request or  not if yes   add  count +1 
                working_list = [] 
                working_list.append(item["name"])
                working_list_url = quote(working_list.__str__())
                URL = "https://graph.facebook.com/v9.0/search?access_token={}&interest_list={}&pretty=0&type={}&limit=100".format(
                    access_token, working_list_url, reqType)
                response = requests.get(URL)
                if response.status_code == 200:
                    done_list.append(item["name"])
                    chunk_done.append({'name':item["name"]})
                    # print(len(done_list) , ' done_list')
                    data = response.json()["data"]
                    for interest in data:
                        if interest not in all_interest_list:
                            all_interest_list.append(interest)
                            chunk_intersts.append(interest)
                else:
                    print('error in response  => \n ' ,' status_code =' ,response.status_code , "   " , response , "   " , URL)
                    break
            count += 1
        else:
          print("start")
          print(len(done_list) ,"   from   " , len(all_interest_list))
          threading.Thread(target=append,args=(chunk_intersts,chunk_done,)).start()
          chunk_intersts = []
          chunk_done = []
          count = 0
    print(len(done_list) ,"   from   " , len(all_interest_list))
    print("start")
    threading.Thread(target=append,args=(chunk_intersts,chunk_done,)).start()
 

In [51]:
get_data()
print("data is ready")
print(f'Done {len(done_list)}  from  {len(all_interest_list)} ')

data is ready
Done 82949  from  82950 


In [39]:
# to save the output in file to use it later
def save_csv():
      with open("./output_data.csv","w") as out_file:
          out_file.write(json.dumps(done_list))

In [ ]:
main()

In [48]:
cleaning_collection(interstdb)

82949


In [49]:
cleaning_collection(donedb)

82949
